<img width="50px" align="left" style="margin-right:20px" src="http://data.cluster-lab.com/public-newprolab-com/npl_logo.png"> <b>New Professions Lab</b> <br /> Специалист по большим данным

# Проект 1

# Спрогнозировать пол и возрастную категорию интернет-пользователей по логу посещения сайтов

<img width="110px" align="left" src="http://data.cluster-lab.com/public-newprolab-com/project01_img0.png?img">

Одна из задач DMP-системы состоит в том, чтобы по разрозненным даннным, таким, как посещения неким пользователем сайтов, классифицировать его и присвоить ему определённую категорию: пол, возраст, интересы и так далее. В дальнейшем составляется портрет, или профиль, пользователя, на основе которого ему более таргетированно показывается реклама в интернете.

### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.

Мы не ограничиваем вас в выборе инструментов и методов работы с данными. Используйте любые эвристики, внешние источники, парсинг контента страниц — всё, что поможет вам выполнить задачу. Единственное ограничение — никаких ручных действий. Руками проставлять классы нельзя.

Поскольку это ваш проект, который мы наверняка захотите показать другим, уделите его оформлению достаточно времени. Мы рекомендуем сделать весь проект в этом ноутбуке. Снизу, под заданием, вы сможете описать ваше решение.

⏰ **Дедлайн: 16 мая 2019, 23:59**

### Это - не обычная лабораторка

Это - соревнование! В нем есть доска лидеров, которую можно увидеть после первой успешной проверки.

Но это так же и не обычное соревнование! Вы не проверяете ваш результат (предсказания), вместо этого проверочный скрипт запускает вашу программу, дает ей на стандартный ввод проверочные данные, и принимает предсказания от вашей программы на стандартном выводе.

Внимательно прочитайте секции "Обработка тестовых данных и формат вывода результатов" и "Проверка" ниже.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
import pickle
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

### Обработка данных на вход

Для выполнения работы вам следует тспользовать файл `/data/share/project01/gender_age_dataset.txt`

In [2]:
file_path = '/data/share/project01/gender_age_dataset.txt'

Он содержит данные о посещении сайтов ~40 000 пользователей, при этом по некоторым из них (~ 35 000) известны их пол и возрастная категория, а по 5 000 - эта информация не известна. В файле есть 4 поля:
* **gender** - пол, принимающий значения `M` (male - мужчина), `F` (female - женщина), `-` (пол неизвестен);
* **age** - возраст, представленный в виде диапазона x-y (строковый тип), или `-` (возрастная категория неизвестна);
* **uid** - идентификатор пользователя, строковая переменная;
* **user_json** - поле json, в котором содержатся записи о посещении сайтов этим пользователем `(url, timestamp)`.

Первое, что обычно делают в таких случаях, — исследуют имеющийся датасет и разбираются, какие же данные мы получили.

Загрузим весь датасет в pandas:

In [3]:
df = pd.read_csv(file_path, sep='\t')

И теперь попробуем понять, что у нас есть:

In [4]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


Что содержится в `user_json`?

In [5]:
df.iloc[0].user_json

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

Видим, что это некая сериализованная json-строка, которую можно легко разобрать через модуль `json`:

In [6]:
json.loads(df.iloc[0].user_json)

{'visits': [{'url': 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun',
   'timestamp': 1419688144068},
  {'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story',
   'timestamp': 1426666298001},
  {'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html',
   'timestamp': 1426666298000},
  {'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story',
   'timestamp': 1426661722001},
  {'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html',
   'timestamp': 1426661722000}]}

Методом `pandas.DataFrame.apply` (хотя не только им) можно применить операцию десериализации json-строк ко всему датасету. Рекомендуем почитать [документацию по методу apply](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

Работая с подобными операциями, обратите внимание на kwargs-аргумент `axis`. Часто, забыв его указать, вы примените операцию не к ряду (строке), а к столбцу, что вряд ли входит в ваши планы.

### Очистка данных и feature engineering

Очистка данных и генерация новых фич составит значительную часть вашей работы. Именно здесь вы и должны продемонстрировать знания и креативность: чем лучше окажутся ваши фичи и чем лучше сможете убрать шум из датасета, тем лучших результатов вы достигнете.

Одна из первых вещей, которые можно попробовать — это вытащить домены и использовать их в качестве признаков. Можно воспользоваться функцией:

In [7]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    if netloc is not None: return str(netloc.encode('utf8')).strip()
    return None

Поскольку эта часть и есть ваша работа, мы не станем раскрывать все секреты (хотя несколько советов мы всё же дали, посмотрите ниже в разделе Подсказки).

### Деление на train и test сеты, обучение модели, предсказания для test-сета

Давайте теперь оценим размер нашего train и test сетов. Train set:

In [8]:
len(df[~((df.gender == '-') & (df.age == '-'))])

36138

Test set:

In [9]:
len(df[(df.gender == '-') & (df.age == '-')])

5000

In [10]:
len(df) # Весь датасет

41138

Когда вы очистили данные и сгенерировали признаки, которые можно дать на вход алгоритму, следующий этап — это разделить данные на тренировочную и тестовую выборки. Сохраните train и test выборки в отдельных файлах, используя метод `pandas.DataFrame.to_csv`. Либо просто сделайте два датафрейма: `train_df` и `test_df`. Обучите модель на ваш выбор, оцените результат, подумайте, как можно его улучшить.

In [ ]:
# ваш код
# p1 = Pipeline(
# ...
# )

###  Обучение модели

Обучите модель на ваш выбор, оцените результат, подумайте, как можно его улучшить.

In [ ]:
# ваш код
# p1.fit(...)

### Сохранение модели

Обучать модель вы можете в ноутбуке - это удобно. А после того как модель обучилась и стала выдавать приемлемое качество на вашей валидационной выборке, сохраните ее в отдельном файле (например, используя pickle):

In [ ]:
#Сохранить модель, которая содержится в переменной vectorizer
#Установите правильные атрибуты файла модели, чтобы он мог быть считан проверочным скриптом:
model_file = "project01_model.pickle"

with open(os.environ['HOME'] + '/' + model_file, 'wb') as f:
    pickle.dump(p1, f)
    
os.chmod(os.environ['HOME'] + '/' + model_file, 0o644)

Однако, не забудьте сохранить и код генерации признаков и обучения модели - это нужно для воспроизводимости результатов.

### Обработка тестовых данных и формат вывода результатов

Помимо того, что у вас должна получиться точная модель, вам нужно уложиться в SLA (service-level agreement). Всё почти как по-настоящему. Результатом вашей работы в данном случае будет не выходной файл, в котором вы всё посчитали для скрытой выборки, а скрипт, который мы будем запускать и проверять SLA и точность.

Создайте в корне своей домашней директории файл `project01_gender-age.py`. 

Назначте ему нужные права: `chmod 755 project01_gender-age.py`.

Вот фрагмент кода, который считывает данные из потока стандартного ввода:

In [19]:
#!/opt/anaconda/envs/bd9/bin/python

import sys
import pandas as pd

columns=['gender','age','uid','user_json']

df = pd.read_table(
    sys.stdin, 
    sep='\t', 
    header=None, 
    names=columns
)

Далее вы должны применить все те же самые преобразования данных, которые (возможно) применили к обучающей выборке.
Затем считываете модель:

In [ ]:
#считать модель из файла в переменную vectorizer
import pickle

model_file = "project01_model.pickle"
p1 = pickle.load(open(model_file, 'rb'))

Путь к модели необходимо указывать относительно вашей домашней директории. Например, если в директории `name.surname` есть папка `project01`, в которой лежит модель `project01_model.pickle`, то путь к модели в скрипте будет выглядеть как `project01/project01_model.pickle`.

Задача вашего скрипта сделать предсказание по всем полученным строкам и выдать результат в формате json. В файле должны быть только те пользователи, у которых пол и возрастная категория изначально неизвестны, и они должны быть **отсортированы по UID по возрастанию значений лексикографически.** Пример вывода указан ниже.

In [ ]:
output = output[['uid', 'gender', 'age']]
output.sort_values(by='uid',axis = 0, ascending = True, inplace = True)
sys.stdout.write(output.to_json(orient='records'))

Для самопроверки вы можете локально оттестировать свой скрипт, используя следующую команду:

In [ ]:
!tail -n1000 gender_age_dataset.txt | ./project01_gender-age.py > output.json

### Подсказки

1. Есть много различных способов решить данную задачу: можно просто хорошо поработать с урлами и доменами, можно пропарсить содержимое этих урлов (заголовки, текст и т.д.) и воспользоваться неким векторизатором типа TF\*IDF для генерации дополнительных фич, которые уже в дальнейшем вы подадите на вход ML-алгоритму, можно сделать тематическое моделирование (LDA, BigARTM) сайтов и использовать одну или несколько тем в качестве фич.

2. Возможно, что данные грязные и их нужно дополнительно обработать. Спецсимволы, кириллические домены? Уделите этому этапу достаточно времени: здесь чистота датасета важнее, чем выбор алгоритма.

3. Часто бывает, что лучшее решение с точки зрения результата — оно же самое простое. Попробуйте сначала простые способы, простые алгоритмы, прежде чем переходить к тяжёлой артиллерии. Один из вариантов — начать с небольшого RandomForest.

4. Вам почти наверняка понадобится что-то из пакета sklearn. [Документация](http://scikit-learn.org/stable/user_guide.html) — ваш лучший друг.

5. Вы можете сначала предсказать пол, а затем возраст, либо сразу и то, и другое. Экспериментируйте.

6. Объединяйтесь в команды. Так гораздо веселее и интереснее.

### Проверка
Проверка осуществляется из [Личного кабинета](http://lk.newprolab.com/lab/project1). До дедлайна вы будете проверять работу своего скрипта на валидационной выборке (2 000 записей). При наступлении дедлайна мы автоматически пересчитаем модели по скрытой тестовой выборке (3 000 записей). Это и будет финальным результатом.

* В поле `part of users with predicted gender + age` - указана доля пользователей, которая была предсказана от общего числа неизвестных пользователей (пример: по 3 000 был сделан прогноз, а всего было неизвестно 5 000, чекер выдаст 0.6).

* В поле `correctly predicted users / total number of users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа всех пользователей (пример: по 3 000 был сделан прогноз, правильно было спрогнозировано 1 500, а всего было неизвестно 5 000, чекер выдаст 0.3)

* В поле `correctly predicted users / number of predicted users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа предсказанных пользователей (пример: по 3 000 был сделан прогноз, из них правильно предсказано 1 500, чекер выдаст 0.5).

**Важное замечание!** Вы должны дать прогноз хотя бы по 50% пользователей, у которых изначально не указан пол и возрастная категория. Иными словами, вы можете оставить неопределенными не более 50% изначально неопределенных пользователей.

**Если доля в последнем поле превысит порог 0.28, то проект будет засчитан, при условии что выполнен SLA в 0.04 секунды на каждого пользователя (т.е. на каждую строчку тестового датасета)**

Лучшей команде, набравшей максимальный результат, мы подарим специальный приз, о котором скажем позднее.